In [1]:
!pip install textattack[tensorflow]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 50.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.3/374.3

In [2]:
# Directory for Colab (optional, use if running Colab)
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
import os
os.chdir('/content/gdrive/MyDrive/NLP_project')

In [4]:
!pwd

/content/gdrive/MyDrive/NLP_project


In [6]:
import pandas as pd
df_train = pd.read_pickle("pickle/imdb_train.pickle")
df_train

,text,label
8305,"With one of my very favorite actors, James Spa...",0
10408,"$25,000 Pyramid Clues: Deep Blue Sea. Tremors....",0
19465,"If you're researching UFO facts, then this vid...",1
8775,This film concerns purportedly non-establishme...,0
16013,Phantom Lady (1944) Dir: Robert Siodmak <br />...,1
...,...,...
6271,"Ironically, although he can do the ""splits"", T...",0
21834,Bo Derek's beauty and John Derek's revolutiona...,1
1115,"OK, so it was written in 1996, before 9/11, so...",0
8107,"Did I miss something here? This ""adaptation"" h...",0


In [7]:
df_test = pd.read_pickle("pickle/imdb_bert_test_textfooler.pickle")
df_test

,text,adversarial
3098,"So, I'm wondering while watching this film, di...",1
288,"Although not a big Coen brothers fan, me am an...",1
1686,me am in a theatres nightclubs at my pupil and...,1
5350,The Thirdly Stooges enjoys always been some of...,1
8615,"(Possible ?? spoilers included, but nothing cr...",1
...,...,...
87,This would probably be a good film to see....p...,0
4274,"I haven't seen a lot of episodes of ""Family Gu...",0
1051,The film was half over before I managed to fig...,0
6107,"When I first saw the movie, I thought it was s...",0


In [8]:
from transformers import BertModel, BertTokenizer

model_name = 'textattack/bert-base-uncased-imdb'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at textattack/bert-base-uncased-imdb were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
from embeddings import get_embeddings
import torch

In [10]:
# Check if you can access to GPU:
torch.cuda.is_available()

True

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [13]:
train_matrix = get_embeddings(df_train['text'], model.cuda(), tokenizer, device)
train_matrix.shape

(8000, 768)

In [14]:
test_matrix = get_embeddings(df_test['text'], model.cuda(), tokenizer, device)
test_matrix.shape

(3680, 768)